In [4]:
def to_df(results_all, answer_type, plan_length, domain):
    d_ramifications_pretty = {
        WITH_RAMIFICATIONS : 'R',
        WITHOUT_RAMIFICATIONS : 'No R'
    }
    d_answer_type_pretty = {
        TRUE_FALSE_ANSWER_TYPE : 'T/F',
        FREE_ANSWER_TYPE : 'Free'
    }
    
    index = []
    data = []
    for question_category in QUESTION_CATEGORIES + [ALL_CATEGORIES_KEY]:
        for ramifications in RAMIFICATION_TYPES:
            index.append((plan_length, question_category, d_ramifications_pretty[ramifications], d_answer_type_pretty[answer_type]))
            data_columns = {}
            for model_name in PROMPT_MODEL_NAMES:
                for prompt_type in PROMPT_TYPES:
                    instance = filter_single_selector(results_all, plan_length, question_category, ramifications, model_name, prompt_type, domain,
                          answer_type, WITHOUT_RANDOM_SUB)
                    if len(instance) == 0:
                        data_columns[(model_name, prompt_type)] = None
                    elif not len(instance) == 1: 
                        raise ValueError(f'len(instance) == {len(instance)}')
                    else:
                        data_columns[(model_name, prompt_type)] = instance[0][SK_RESULT]
            data.append(data_columns)
    return pd.DataFrame(data, index = index)

def df_to_latex_table(df):
        latex_table = df.to_latex(index=True, formatters={"name": str.upper}, float_format="{:.2f}".format)
        return latex_table[latex_table.find('\midrule'):].replace("bottomrule", 'crap').replace("\crap", '').replace("\end{tabular}", '')

def assemble_table(results_all, answer_type, domain, score_key=None):
    latex_table_all = ''
    with open('latex_table_template/top') as f:
        latex_table_all += f.read() + '\n'
    latex_table_all += '\n'.join([df_to_latex_table(to_df(results_all, answer_type, plan_length, domain)) for plan_length in PLAN_LENGTHS])
    with open('latex_table_template/bottom') as f:
        latex_table_all += f.read()
    
    caption = f'{answer_type}, {score_key} scores for {domain}'.replace('_', ' ')
    latex_table_all = latex_table_all.replace('REPLACE_CAPTION_KEY', caption)
    
    return latex_table_all


In [5]:
import sys
import json
sys.path.append('..')

from src.common import *
from model_performances import *
from copy import deepcopy

import pandas as pd

EVERYTHING_POSTFIX = 'everything'

In [6]:
def collect_results_all(answer_response_type):
    results_all = []
    for domain, plan_length, question_category, ramifications, random_sub, model_name, prompt_type in for_loop_it():
        dir = results_data_path(answer_response_type, domain, plan_length, question_category, ramifications, random_sub, model_name, prompt_type)
        path = os.path.join(dir, RESULTS_FILE_NAME)
        if os.path.exists(path):
            with open(path) as f:
                results_all.append(json.load(f))
    return results_all


In [8]:
answer_type = TRUE_FALSE_ANSWER_TYPE #FREE_ANSWER#
score_key = F1_SCORE_KEY#'accuracy'
results_all = collect_results_all(tf_answer_type(score_key = F1_SCORE_KEY))
# results_all = open_jsonl(os.path.join(STATISTICS_PATH, save_stats_file(answer_type, score_key)))

domain = ALL_DOMAINS_KEY

# for domain in DOMAIN_NAMES + [ALL_DOMAINS_KEY]:
#     latex_table_all = assemble_table(results_all, answer_type, domain, score_key=score_key)
#     os.makedirs(os.path.join(STATISTICS_PATH, 'tables'), exist_ok=True)
#     with open(os.path.join(STATISTICS_PATH, 'tables', f'{domain}.{answer_type}.{score_key}.tex'), 'w') as f:
#         f.write(latex_table_all)

plan_length = 1
df = to_df(results_all, answer_type, plan_length, domain)


100%|██████████| 16128/16128 [00:01<00:00, 8974.72it/s] 


In [9]:
df

,"(gemma-2b-it, few_shot_1)","(gemma-2b-it, few_shot_3)","(gemma-2b-it, few_shot_5)","(Llama-2-7b-chat-hf, few_shot_1)","(Llama-2-7b-chat-hf, few_shot_3)","(Llama-2-7b-chat-hf, few_shot_5)"
"(1, object_tracking, R, T/F)",0.031156,None,None,0.167051,0.294849,0.377315
"(1, object_tracking, No R, T/F)",0.022500,None,None,0.169412,0.281915,0.399538
"(1, fluent_tracking, R, T/F)",0.161350,None,None,0.286730,0.317757,0.344178
"(1, fluent_tracking, No R, T/F)",0.155067,None,None,0.293312,0.314741,0.355172
"(1, state_tracking, R, T/F)",0.405000,None,None,0.428571,1.000000,NaN
"(1, state_tracking, No R, T/F)",0.400990,None,None,0.392857,0.500000,NaN
"(1, action_executability, R, T/F)",0.128906,None,None,0.221311,0.191489,0.270270
"(1, action_executability, No R, T/F)",0.125969,None,None,0.179167,0.191489,0.229730
"(1, effects, R, T/F)",0.029768,None,None,0.161483,0.252427,0.382231
"(1, effects, No R, T/F)",0.023001,None,None,0.174390,0.257282,0.373967


In [7]:
results_all[0]

[{'result': 0.08571428571428572,
  'model': 'Llama-2-7b-chat-hf',
  'prompt_type': 'few_shot_1',
  'ramification_type': 'with_ramifications',
  'substitution_type': 'with_random_sub',
  'domain_name': 'blocksworld',
  'plan_length': 1,
  'question_category': 'object_tracking',
  'answer_type': 'true_false_answer'}]